In [1]:
import pandas as pd
import numpy as np
import glob
import time

train_df = pd.read_csv('../../datasets/Zenodo/v1/training_df_single_fold.csv.gz')
train_smiles = train_df['rdkit SMILES'].tolist()
ams_df = pd.read_csv('../../datasets/Zenodo/v1/ams_order_results.csv.gz')
ams_smiles = ams_df['rdkit SMILES'].tolist()

rf_pred_files = glob.glob('L:/Data/datasets/REAL_db_rf_preds/*.csv.gz')

simbl_pred_files = glob.glob('L:/Data/datasets/REAL_db_simbaseline_preds/*.csv.tar.gz')

In [2]:
topk = 110000

simbaseline_topdf = None
for pred_file in simbl_pred_files:
    start_time = time.time()
    pred_df = pd.read_csv(pred_file)
    pred_df = pred_df.sort_values('simbaseline_preds', ascending=False)

    tmp_df = pred_df.iloc[:topk,:]

    if simbaseline_topdf is None:
        simbaseline_topdf = tmp_df
    else:
        simbaseline_topdf = pd.concat([simbaseline_topdf, tmp_df], axis=0)
        simbaseline_topdf = simbaseline_topdf.sort_values('simbaseline_preds', ascending=False).iloc[:topk,:]
        
    print('File: {}. Total time: {} minutes'.format(pred_file.split('\\')[-1],
                                                                (time.time() - start_time)/60.0))

File: simbaseline_preds_part_00_process_0.csv.tar.gz. Total time: 1.7032963116963704 minutes
File: simbaseline_preds_part_01_process_0.csv.tar.gz. Total time: 1.7733971198399863 minutes
File: simbaseline_preds_part_02_process_0.csv.tar.gz. Total time: 1.784189514319102 minutes
File: simbaseline_preds_part_03_process_0.csv.tar.gz. Total time: 1.738530421257019 minutes
File: simbaseline_preds_part_04_process_0.csv.tar.gz. Total time: 1.7609183033307394 minutes
File: simbaseline_preds_part_05_process_0.csv.tar.gz. Total time: 1.7519999106725057 minutes
File: simbaseline_preds_part_06_process_0.csv.tar.gz. Total time: 1.742218049367269 minutes
File: simbaseline_preds_part_07_process_0.csv.tar.gz. Total time: 1.7827510714530945 minutes
File: simbaseline_preds_part_08_process_0.csv.tar.gz. Total time: 1.7447100758552552 minutes
File: simbaseline_preds_part_09_process_0.csv.tar.gz. Total time: 1.7621355811754862 minutes
File: simbaseline_preds_part_10_process_0.csv.tar.gz. Total time: 1.75354

In [6]:
randforest_topdf = None
for pred_file in rf_pred_files:
    start_time = time.time()
    pred_df = pd.read_csv(pred_file)
    pred_df.columns = ['smiles', 'rf_preds']
    pred_df = pred_df.sort_values('rf_preds', ascending=False)

    tmp_df = pred_df.iloc[:topk,:]
    if randforest_topdf is None:
        randforest_topdf = tmp_df
    else:
        randforest_topdf = pd.concat([randforest_topdf, tmp_df], axis=0)
        randforest_topdf = randforest_topdf.sort_values('rf_preds', ascending=False).iloc[:topk,:]
        
    print('File: {}. Total time: {} minutes'.format(pred_file.split('\\')[-1],
                                                                (time.time() - start_time)/60.0))

File: rf_preds_part_00_process_0.csv.gz. Total time: 1.283363934357961 minutes
File: rf_preds_part_01_process_0.csv.gz. Total time: 1.2706236044565837 minutes
File: rf_preds_part_02_process_0.csv.gz. Total time: 1.2707703351974486 minutes
File: rf_preds_part_03_process_0.csv.gz. Total time: 1.2597245852152505 minutes
File: rf_preds_part_04_process_0.csv.gz. Total time: 1.2472380677858987 minutes
File: rf_preds_part_05_process_0.csv.gz. Total time: 1.248350195089976 minutes
File: rf_preds_part_06_process_0.csv.gz. Total time: 1.2397375663121541 minutes
File: rf_preds_part_07_process_0.csv.gz. Total time: 1.2686275442441304 minutes
File: rf_preds_part_08_process_0.csv.gz. Total time: 1.2464696923891703 minutes
File: rf_preds_part_09_process_0.csv.gz. Total time: 1.2777347286542258 minutes
File: rf_preds_part_10_process_0.csv.gz. Total time: 1.278027097384135 minutes
File: rf_preds_part_11_process_0.csv.gz. Total time: 1.3076749801635743 minutes
File: rf_preds_part_Premium_process_0.csv.g

In [8]:
"""
Compute PAINS Filter.
Then remove enamine cpds that are also in training or ams datasets.
"""

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem.FilterCatalog import *

params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS_A)
params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS_B)
params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS_C)
pains_catalog = FilterCatalog(params)
FP_radius=2
FP_size=1024
saltRemover = SaltRemover(defnFilename='../../datasets/patterns/Salts.txt')

# simbaseline
rdkit_mols = simbaseline_topdf['smiles'].astype(str).apply((lambda x: Chem.MolFromSmiles(x)))
rdkit_mols = rdkit_mols.apply((lambda x: saltRemover.StripMol(x)))
simbaseline_topdf['rdkit SMILES'] = rdkit_mols.apply((lambda x: Chem.MolToSmiles(x)))
simbaseline_topdf['1024 MorganFP Radius 2'] = rdkit_mols.apply((lambda x: AllChem.GetMorganFingerprintAsBitVect(x, 
                                                                                       radius=FP_radius, 
                                                                                       nBits=FP_size).ToBitString()))

simbaseline_topdf['PAINS Filter'] = rdkit_mols.apply((lambda x: not pains_catalog.HasMatch(x))).astype(int)
simbaseline_topdf = simbaseline_topdf[~simbaseline_topdf['rdkit SMILES'].isin(train_smiles)]
simbaseline_topdf = simbaseline_topdf[~simbaseline_topdf['rdkit SMILES'].isin(ams_smiles)]

# rf
rdkit_mols = randforest_topdf['smiles'].astype(str).apply((lambda x: Chem.MolFromSmiles(x)))
rdkit_mols = rdkit_mols.apply((lambda x: saltRemover.StripMol(x)))
randforest_topdf['rdkit SMILES'] = rdkit_mols.apply((lambda x: Chem.MolToSmiles(x)))
randforest_topdf['1024 MorganFP Radius 2'] = rdkit_mols.apply((lambda x: AllChem.GetMorganFingerprintAsBitVect(x, 
                                                                                       radius=FP_radius, 
                                                                                       nBits=FP_size).ToBitString()))

randforest_topdf['PAINS Filter'] = rdkit_mols.apply((lambda x: not pains_catalog.HasMatch(x))).astype(int)
randforest_topdf = randforest_topdf[~randforest_topdf['rdkit SMILES'].isin(train_smiles)]
randforest_topdf = randforest_topdf[~randforest_topdf['rdkit SMILES'].isin(ams_smiles)]

<frozen importlib._bootstrap>:228: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterCatalogEntry const > already registered; second conversion method ignored.


---
# Check overlap of RF and SimBaseline

Perform similar filtering steps to get down to 100 cpds.

In [151]:
topk_list = [50, 100, 250, 500, 1000, 5000, 10000, 
             20000, 30000, 40000, 50000, 
             60000, 70000, 80000, 90000, 100000]

simbaseline_topdf = simbaseline_topdf.sort_values('simbaseline_preds', ascending=False)
randforest_topdf = randforest_topdf.sort_values('rf_preds', ascending=False)

data = []
for topk in topk_list:
    tmp_simbaseline = simbaseline_topdf.iloc[:topk,:]
    tmp_simbaseline_smiles = tmp_simbaseline['rdkit SMILES'].tolist()
    
    tmp_randforest = randforest_topdf.iloc[:topk,:]
    tmp_randforest_smiles = tmp_randforest['rdkit SMILES'].tolist()
    
    overlap_count = np.intersect1d(tmp_simbaseline_smiles, tmp_randforest_smiles).shape[0]
    overlap_perc = round(100.0*overlap_count/topk, 2)
    
    data.append([topk, overlap_count, overlap_perc])
    
overlap_df = pd.DataFrame(data=data, columns=['TopK', 'SimBaseline vs RF Overlap Count', 'SimBaseline vs RF Overlap %'])

In [152]:
overlap_df

,TopK,SimBaseline vs RF Overlap Count,SimBaseline vs RF Overlap %
0,50,17,34.00
1,100,25,25.00
2,250,65,26.00
3,500,120,24.00
4,1000,220,22.00
5,5000,936,18.72
6,10000,2013,20.13
7,20000,4414,22.07
8,30000,6652,22.17
9,40000,8728,21.82


In [155]:
scrutinized_df = pd.read_csv('./scrutinized_df.csv')
scrutinized_smiles = scrutinized_df[scrutinized_df['Scrutinized Hit'] == 1]['rdkit SMILES'].tolist()
topk_list = [50, 100, 250, 500, 1000, 5000, 10000, 
             20000, 30000, 40000, 50000, 
             60000, 70000, 80000, 90000, 100000]

simbaseline_topdf = simbaseline_topdf.sort_values('simbaseline_preds', ascending=False)
randforest_topdf = randforest_topdf.sort_values('rf_preds', ascending=False)

data = []
for topk in topk_list:
    tmp_simbaseline = simbaseline_topdf.iloc[:topk,:]
    tmp_simbaseline_smiles = tmp_simbaseline['rdkit SMILES'].tolist()
    
    overlap_count = np.intersect1d(tmp_simbaseline_smiles, scrutinized_smiles).shape[0]
    overlap_perc = round(100.0*overlap_count/topk, 2)
    
    data.append([overlap_count, overlap_perc])
    
overlapping_with_hits_df = pd.DataFrame(data=data, columns=['28 Confirmed Hit Overlap Count', '28 Confirmed Hit Overlap %'])

In [156]:
pd.concat([overlap_df, overlapping_with_hits_df], axis=1)

,TopK,SimBaseline vs RF Overlap Count,SimBaseline vs RF Overlap %,28 Confirmed Hit Overlap Count,28 Confirmed Hit Overlap %
0,50,17,34.00,0,0.00
1,100,25,25.00,0,0.00
2,250,65,26.00,0,0.00
3,500,120,24.00,0,0.00
4,1000,220,22.00,0,0.00
5,5000,936,18.72,0,0.00
6,10000,2013,20.13,0,0.00
7,20000,4414,22.07,3,0.01
8,30000,6652,22.17,4,0.01
9,40000,8728,21.82,8,0.02


In [31]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances, pairwise_distances_argmin
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, '../enamine_final_list/rd_filters/') #https://github.com/PatWalters/rd_filters
from rd_filters import *


alert_file_name = '../enamine_final_list/rd_filters/data/alert_collection.csv'
rules_file_name = '../enamine_final_list/rd_filters/data/rules.json'
rf = RDFilters(alert_file_name)
rules_file_path = get_config_file(rules_file_name, "FILTER_RULES_DATA")
rule_dict = read_rules(rules_file_path)
rule_list = [x.replace("Rule_", "") for x in rule_dict.keys() if x.startswith("Rule") and rule_dict[x]]
rule_str = " and ".join(rule_list)
print(f"Using alerts from {rule_str}")
rf.build_rule_list(rule_list)

tmp_df = simbaseline_topdf.copy()
tmp_df['idnumber'] = tmp_df.index.tolist()
input_data = tmp_df[["rdkit SMILES", "idnumber"]].values.tolist()

start_time = time.time()
p = Pool(1)
filter_res = list(p.map(rf.evaluate, input_data))
rd_df = pd.DataFrame(filter_res, columns=["SMILES", "idnumber", "RD_FILTER", "MW", "LogP", "HBD", "HBA", "TPSA"])
filter_binary = np.zeros((rd_df.shape[0],), dtype='uint8')
for index, row in rd_df.iterrows():
    if row['RD_FILTER'] == "OK":
        filter_binary[index] = 1
rd_df["RD_FILTER"] = filter_binary
rd_df = rd_df[["idnumber", "RD_FILTER", "LogP"]]

tmp_df = tmp_df.merge(rd_df, on='idnumber')

Using alerts from Inpharmatica


In [34]:
des_cols = ['rdkit SMILES', 'simbaseline_preds', 'PAINS Filter', 
            '1024 MorganFP Radius 2', "RD_FILTER"]
df = tmp_df[des_cols].sort_values('simbaseline_preds', ascending=False)
df['pred_rank'] = df['simbaseline_preds'].rank(method='first', ascending=False)

In [93]:
cluster_df = pd.read_csv('../../datasets/Zenodo/v1/train_ams_order_cluster.csv.gz')

leaders_idx = cluster_df['TB_0.4 Leader'].unique().tolist()
leaders_df = cluster_df[cluster_df['Index ID'].isin(leaders_idx)]
leaders_df = leaders_df.reset_index(drop=True)

top_tenk = df.iloc[:10000,:]

X_train = np.vstack([np.fromstring(x, 'u1') - ord('0') for x in leaders_df['1024 MorganFP Radius 2']]).astype(float)
X_simbaseline = np.vstack([np.fromstring(x, 'u1') - ord('0') for x in top_tenk['1024 MorganFP Radius 2']]).astype(float)

In [94]:
"""
    Compute tanimoto distance matrix between top_tenk for simbaseline and train+ams cluster leaders
"""

n_features = 1024

numerator = np.dot(X_simbaseline, X_train.T)
denominator = n_features - np.dot(1-X_simbaseline, (1-X_train).T)
td_matrix = numerator / denominator
td_matrix = 1 - td_matrix
nearest_leaders = np.argmin(td_matrix, axis=1)

In [123]:
"""
    Compute tanimoto sim matrix between top_tenk for simbaseline and train and ams ACTIVES only
"""
train_actives_df = cluster_df[(cluster_df['dataset'] == 'train') & (cluster_df['Hit'] == 1)]
ams_actives_df = cluster_df[(cluster_df['dataset'] == 'ams') & (cluster_df['Hit'] == 1)]

X_train = np.vstack([np.fromstring(x, 'u1') - ord('0') for x in train_actives_df['1024 MorganFP Radius 2']]).astype(float)
n_features = 1024
numerator = np.dot(X_simbaseline, X_train.T)
denominator = n_features - np.dot(1-X_simbaseline, (1-X_train).T)
td_matrix = numerator / denominator
td_matrix = 1 - td_matrix
nearest_train_active_tandist = np.min(td_matrix, axis=1)

X_train = np.vstack([np.fromstring(x, 'u1') - ord('0') for x in ams_actives_df['1024 MorganFP Radius 2']]).astype(float)
n_features = 1024
numerator = np.dot(X_simbaseline, X_train.T)
denominator = n_features - np.dot(1-X_simbaseline, (1-X_train).T)
td_matrix = numerator / denominator
td_matrix = 1 - td_matrix
nearest_ams_active_tandist = np.min(td_matrix, axis=1)

In [129]:
top_tenk = df.iloc[:10000,:]
tmp_df = leaders_df.iloc[nearest_leaders,:][['TB_0.2 ID','TB_0.2 Leader', 'TB_0.3 ID', 'TB_0.3 Leader', 'TB_0.4 ID', 'TB_0.4 Leader', 'dataset', 'Hit']]
top_tenk = pd.concat([top_tenk.reset_index(drop=True), tmp_df.reset_index(drop=True)], axis=1)

top_tenk['Is Train active cluster?'] = ((top_tenk['dataset'] == 'train') & (top_tenk['Hit'] == 1)).astype(int)
top_tenk['Is AMS active cluster?'] = ((top_tenk['dataset'] == 'ams') & (top_tenk['Hit'] == 1)).astype(int)

top_tenk['Closest Train Active TanDist'] = nearest_train_active_tandist
top_tenk['Closest AMS Active TanDist'] = nearest_ams_active_tandist

In [136]:
print('Shape: {}'.format(top_tenk.shape[0]))
# get cpds in clusters that don't exist in train or ams ACTIVE clusters
prune_df1 = top_tenk[(top_tenk['Is Train active cluster?'] == 0) & (top_tenk['Is AMS active cluster?'] == 0)]
print('1. Shape: {}'.format(prune_df1.shape[0]))

# pass PAINS and RD_FILTERS filter
prune_df2 = prune_df1[(prune_df1['PAINS Filter'] == 1) * (prune_df1['RD_FILTER'] == 1)]
print('2. Shape: {}'.format(prune_df2.shape[0]))

# take only cpds that are from from tain and ams actives
tandist_thresh=0.335
prune_df3 = prune_df2[(prune_df2['Closest Train Active TanDist'] >= tandist_thresh) & (prune_df2['Closest AMS Active TanDist'] >= tandist_thresh)]
print('3. Shape: {}'.format(prune_df3.shape[0]))

# get highest prediction from each cluster
prune_df4 = prune_df3.drop_duplicates(subset='TB_0.4 ID', keep='first').reset_index(drop=True)
print('4. Shape: {}'.format(prune_df4.shape[0]))

Shape: 10000
1. Shape: 7411
2. Shape: 5906
3. Shape: 1023
4. Shape: 311


In [142]:
num_cpds = 100
cpds_to_select = [0] # first select cpd with highest rf_rank

X_prosp = np.vstack([np.fromstring(x, 'u1') - ord('0') for x in prune_df4['1024 MorganFP Radius 2']]).astype(float)
for i in range(1, 100):
    x = X_prosp[cpds_to_select,:]
    remaining_cpds = np.setdiff1d(np.arange(X_prosp.shape[0]), cpds_to_select)
    y = X_prosp[remaining_cpds,:]
    tandist = pairwise_distances(y, x, metric='jaccard')
    farthest_idx = np.argmax(tandist.mean(axis=1)); 
    
    cpds_to_select.append(remaining_cpds[farthest_idx])
    
simbaseline_final_list = prune_df4.iloc[cpds_to_select,:]
simbaseline_final_list = simbaseline_final_list.sort_values('simbaseline_preds', ascending=False)

In [147]:
rf_final_list = pd.read_csv('../../datasets/enamine_final_list_v2.csv.gz')

In [148]:
tmp_simbaseline_smiles = simbaseline_final_list['rdkit SMILES'].tolist()
tmp_rf_smiles = rf_final_list['rdkit SMILES'].tolist()

overlap_count = np.intersect1d(tmp_simbaseline_smiles, tmp_rf_smiles).shape[0]
overlap_perc = round(100.0*overlap_count/topk, 2)

print('Overlap of Enamine final 100 list SimBaseline vs RF. Count: {}. Perc: {}%'.format(overlap_count, overlap_perc))

overlap_count = np.intersect1d(tmp_simbaseline_smiles, scrutinized_smiles).shape[0]
overlap_perc = round(100.0*overlap_count/topk, 2)

print('Overlap of Enamine final 100 list SimBaseline vs Confirmed hits. Count: {}. Perc: {}%'.format(overlap_count, overlap_perc))

Overlap of Enamine final 100 list SimBaseline vs RF. Count: 0. Perc: 0.0%
Overlap of Enamine final 100 list SimBaseline vs Confirmed hits. Count: 0. Perc: 0.0%


In [29]:
import glob 
import pandas as pd
import numpy as np

log_dir = 'L:/Data/datasets/REAL_db_simbaseline_preds/logs/*.out'
logs = glob.glob(log_dir)

datalist = []
for lf in logs:
    with open(lf, 'r') as f:
        content = f.readlines()
    
    timeline = [l for l in content if 'Total time' in l][0].split(' ')
    filename = [l for l in content if './simbaseline_preds_' in l][0][2:][:-5]
    
    minutes = float(timeline[2])
    hours = minutes / 60.0
    cpd_count = int(timeline[-2])
    
    datalist.append([filename, minutes, hours, cpd_count])


time_df = pd.DataFrame(datalist, 
                       columns=['File', 'Minutes', 'Hours', '# compounds']).sort_values('File')

time_df['Part'] = np.arange(time_df.shape[0])
time_df = time_df.append(pd.DataFrame([['mean', time_df['Minutes'].mean(),  time_df['Hours'].mean(), time_df['# compounds'].mean(), 'mean']], 
                            columns=time_df.columns))
time_df = time_df.append(pd.DataFrame([['std', time_df['Minutes'].std(),  time_df['Hours'].std(), time_df['# compounds'].std(), 'std']], 
                            columns=time_df.columns))

time_df['Hours'] = time_df['Hours'].round(decimals=2)
time_df['Minutes'] = time_df['Minutes'].round(decimals=2)
time_df

C:\Users\Moeman\AppData\Local\Temp\ipykernel_5180\1600154431.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(pd.DataFrame([['mean', time_df['Minutes'].mean(),  time_df['Hours'].mean(), time_df['# compounds'].mean(), 'mean']],
C:\Users\Moeman\AppData\Local\Temp\ipykernel_5180\1600154431.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(pd.DataFrame([['std', time_df['Minutes'].std(),  time_df['Hours'].std(), time_df['# compounds'].std(), 'std']],


,File,Minutes,Hours,# compounds,Part
3,simbaseline_preds_part_00_process_0,1132.34,18.87,6.026253e+07,0
4,simbaseline_preds_part_01_process_0,1032.87,17.21,6.026253e+07,1
5,simbaseline_preds_part_02_process_0,1174.46,19.57,6.026253e+07,2
6,simbaseline_preds_part_03_process_0,1104.19,18.40,6.026253e+07,3
7,simbaseline_preds_part_04_process_0,1213.89,20.23,6.026253e+07,4
8,simbaseline_preds_part_05_process_0,1214.83,20.25,6.026253e+07,5
9,simbaseline_preds_part_06_process_0,1081.04,18.02,6.026253e+07,6
10,simbaseline_preds_part_07_process_0,1212.42,20.21,6.026253e+07,7
11,simbaseline_preds_part_08_process_0,1245.78,20.76,6.026253e+07,8
0,simbaseline_preds_part_09_process_0,1263.50,21.06,6.026253e+07,9


In [30]:
x = time_df[['Part', 'Hours', '# compounds']].copy()
x['# compounds'] = x['# compounds'].astype(int)
print(x.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
Part &  Hours &  \# compounds \\
\midrule
   0 &  18.87 &     60262531 \\
   1 &  17.21 &     60262531 \\
   2 &  19.57 &     60262531 \\
   3 &  18.40 &     60262531 \\
   4 &  20.23 &     60262531 \\
   5 &  20.25 &     60262531 \\
   6 &  18.02 &     60262531 \\
   7 &  20.21 &     60262531 \\
   8 &  20.76 &     60262531 \\
   9 &  21.06 &     60262531 \\
  10 &  22.24 &     60262531 \\
  11 &  17.68 &     60262530 \\
  12 &  19.99 &     60263531 \\
  13 &  20.71 &     60263531 \\
  14 &  14.79 &     60263531 \\
  15 &  20.01 &     60263531 \\
  16 &  19.98 &     60263531 \\
  17 &  20.58 &     53094961 \\
mean &  19.48 &     59864610 \\
 std &   1.68 &      1641881 \\
\bottomrule
\end{tabular}



C:\Users\Moeman\AppData\Local\Temp\ipykernel_5180\1372981113.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(x.to_latex(index=False))
